In [1]:
import xml.etree.ElementTree as ET
import os

def simple_replace_all(root, old_text, new_text):
    """Replace old_text with new_text in all attributes and text content"""
    for elem in root.iter():
        # Replace in all attributes
        for attr_name in elem.attrib:
            if old_text == elem.attrib[attr_name]:
                elem.attrib[attr_name] = elem.attrib[attr_name].replace(old_text, new_text)

folders = ['active_yaw_microtaur', 'active_twist_microtaur', 'rigid_microtaur', 'passive_twist_microtaur']
suffix = ['aym', 'atm', 'rm', 'ptm']

for i, folder in enumerate(folders):
    tree = ET.parse(f"{folder}/robot.xml")
    root = tree.getroot()

    tree_dict = {
        'bodies' : [],
        'joints' : [],
        'positions' : [],
        'classes' : [],
        'sites' : [],
        'meshes' : [],
        'stls' : [],
        'materials' : [],
    }

    for body in root.findall('.//body'):
        if body.get('name') is not None:
            tree_dict['bodies'].append(body.get('name'))

    for joint in root.findall('.//joint'):
        if joint.get('name') is not None:
            tree_dict['joints'].append(joint.get('name'))

    for position in root.findall('.//position'):
        if position.get('name') is not None:
            tree_dict['positions'].append(position.get('name'))

    for default in root.findall('.//default'):
        if default.get('class') is not None:
            tree_dict['classes'].append(default.get('class'))

    for site in root.findall('.//site'):
        if site.get('name') is not None:
            tree_dict['sites'].append(site.get('name'))

    for mesh in root.findall('.//mesh'):
        if mesh.get('file') is not None:
            tree_dict['meshes'].append(mesh.get('file')[:-4])  # Remove the '.stl' extension
            tree_dict['stls'].append(mesh.get('file'))

    for material in root.findall('.//material'):
        if material.get('name') is not None:
            tree_dict['materials'].append(material.get('name'))

    # loop through meshes, find them, copy, and add suffix
    for mesh in tree_dict['stls']:
        mesh_path = f"{folder}/assets/{mesh}"
        new_mesh_path = f"assets/{folder}/{mesh[:-4]}_{suffix[i]}.stl"

        os.makedirs(os.path.dirname(new_mesh_path), exist_ok=True)
        with open(mesh_path, 'rb') as f:
            data = f.read()
        with open(new_mesh_path, 'wb', ) as f:
            f.write(data)

    for lst in tree_dict.values():
        for elem in lst:
            new_name = f"{elem}_{suffix[i]}"
            if ".stl" in elem:
                new_name = f"{elem[:-4]}_{suffix[i]}.stl"
            simple_replace_all(root, elem, new_name)

    # Save the modified XML
    tree.write(f"{folder}/robot_modified.xml", encoding='utf-8', xml_declaration=True)